<h1>CIFAR10 mit Convolutional Neural Network</h1>

<p>Die bisherigen Netzwerke waren &uuml;blicherweise in wenigen Sekunden oder einigen Minute trainierbar und mit der richtigen Netzwerk Struktur waren Genauigkeiten von &uuml;ber 90% zu erreichen. Das wird sich in dieser &Uuml;bung &auml;ndern, da wir das CIFAR10 Datenset verwenden werden. Es enth&auml;lt 50k Trainings- und 10k Testbilder mit je 32x32 RGB-Pixeln. Wie der Name schon sagt, besteht es aus 10 Kategorien. Auch wenn es nur wenige Bildtypen beinhaltet, ist es dennoch sehr kompliziert und wird in der Forschung eingesetzt um neue Ideen zu testen. <a href="http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130" target="_blank">http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130</a></p>


<h3>Vorbereitung</h3>

<p>F&uuml;r das neue Datenset wurde der Programmcode in der <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung4/cvutils.py" target="_blank">cvutils.py</a> Datei aktualisiert. Kopieren Sie sich den Inhalt oder &uuml;berschreiben Sie ihre aktuelle Version mit der Online-Variante. &Uuml;berf&uuml;hren Sie danach das Convolutional Neural Network der letzten &Uuml;bung in die neue Vorlage und passen Sie eventuell die Netzwerkgr&ouml;&szlig;en an. Trainieren Sie einmalig ein paar Iterationen und notieren Sie sich die Genauigkeit ihres ersten Versuches.</p>

<p>Das komplette Notebook steht hier zum&nbsp;<a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung4/Tensorflow_ConvNet_CIFAR10_Vorlage.ipynb" target="_blank">download</a>&nbsp;bereit.</p>

<h3>Aufgabe</h3>

<p>Wird das Netzwerk der letzten Woche &uuml;bernommen, sind Genauigkeiten von knapp 70% zu erwarten. Danach tritt Overfitting ein. Schauen Sie sich dazu die Trainingsfehler und die Genauigkeitskurven an. Ziel der &Uuml;bung ist es, mit einen selbst geschriebenen Neuronalen Netzwerk in so kurzer Zeit wie m&ouml;glich, eine gute Vorhersagegenauigkeit f&uuml;r das neue Daten Set zu erreichen. Ihnen sind keine Grenzen gesetzt, welche Verfahren Sie dabei nutzen. Erstrebenswert sind Genauigkeiten um die 80%. Hier sind ein paar Tipps, wie Sie dies erreichen k&ouml;nnen:</p>

<ul>
	<li>
	<p><strong>Dropout</strong>: Mit Hilfe von Dropout (<a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/dropout" target="_blank">tf.layers.dropout</a>) k&ouml;nnen Sie das Overfitting reduzieren.&nbsp;</p>
	</li>
	<li>
	<p><strong>Weight Regularization</strong>: Die Filterkernel Regulartoren (von <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/conv2d" target="_blank">tf.layers.conv2d</a>&nbsp;und <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/dense" target="_blank">tf.layers.dense</a>) k&ouml;nnen daf&uuml;r sorgen, dass die Gewichtwerte keine extrem gro&szlig;en Werte annehmen.&nbsp;</p>
	</li>
	<li>
	<p><strong>Data Augmentation</strong>: Um mehr Variation in den Trainingsdaten zu haben, k&ouml;nnen diese erweitert (augmentiert) werden. Es gibt viele Funktionen um zuf&auml;llige leichte Ver&auml;nderungen an den Bildern (<a href="https://www.tensorflow.org/api_docs/python/tf/image" target="_blank">tf.image</a>) vornehmen zu k&ouml;nnen.</p>
	</li>
	<li>
	<p><strong>Batch normalization</strong>: Indem das Netzwerk &uuml;berall mit normalisieren Daten (<a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/batch_normalization" target="_blank">tf.nn.batch_normalization</a>) arbeitet, kann es sich besser auf das Wesentliche (Klassifzieren) konzentieren.</p>
	</li>
	<li>
	<p><strong>Netzwerkstruktur</strong>: Die Anzahl und Gr&ouml;&szlig;e der Filterkernel in einem ConvNet entscheiden dar&uuml;ber wie viele verschiedene komplexe Bildmuster das Netzwerk erkennen kann. Inspirieren Sie sich daher bei der Netzwerkstruktur von&nbsp;dem Paper <a href="https://arxiv.org/pdf/1412.6806.pdf" target="_blank">&quot;Striving for Simplicity: The all Convolutional Net&quot;</a>.</p>
	</li>
</ul>

<p>Notieren Sie f&uuml;r Ihr bestes Netzwerk die Genauigkeit und schreiben Sie diese, inklusive dem Netzwerkaufbau in eine PDF Datei. Der strukturelle Aufbau kann in Textform oder visuell festgehalten werden.</p>


In [1]:
import tensorflow as tf
import numpy as np
from cvutils import fetch_cifar10
import math

# load CIFAR10 data set
cifar = fetch_cifar10()
X_train = cifar.train.data.astype('float32')
y_train = cifar.train.target.astype('int64')
X_test = cifar.test.data.astype('float32')
y_test = cifar.test.target.astype('int64')

print(X_train.shape)
print(X_test.shape)

Download MNIST to /Users/sonynka/.keras/datasets
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [2]:
# normalize
X_train = (X_train / 255.) 
X_test = (X_test / 255.)

In [3]:
# TODO Copy the neural network code and implement a convolutional version. Try to reach an accuracy over 96%.
num_classes = 10
kernel_size = 5
num_channels = 1
# Number of channels by layer
n_channels_01 = 6
n_channels_02 = 16
n_channels_03 = 120
n_channels_04 = 84
n_channels_05 = num_classes 

# learn rate
learning_rate = 0.005

batch_size = 32
num_iters = math.ceil(X_train.shape[0] / batch_size)

# Seed
seed = 5
img_dim = 32


graphCNN = tf.Graph()
with graphCNN.as_default():
    x_input = tf.placeholder(dtype=tf.float32, shape=[None, img_dim, img_dim, 3], name='x')
    y_input = tf.placeholder(tf.int64, shape=[None, 1], name='y')
    print('Input shape', x_input.shape)

    # layer 1: output feature maps: 6 filters X 14x14;
    W1 = tf.get_variable("W1", [kernel_size, kernel_size, 3, n_channels_01], 
                         initializer=tf.contrib.layers.xavier_initializer(seed=seed))
    Z1 = tf.nn.conv2d(x_input, W1, strides=[1,1,1,1], padding='VALID', name='conv1')
    A1  = tf.nn.relu(Z1, name='relu1')
    print("A1 shape:", A1.shape)
    P1 = tf.nn.max_pool(A1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='max_pool1')
    print("P1 shape:", P1.shape)
    # 14x14

    # Layer 02 Conv: output feature maps: 16 filters X 10x10
    W2 = tf.get_variable("W2", [kernel_size, kernel_size, n_channels_01, n_channels_02], 
                         initializer=tf.contrib.layers.xavier_initializer(seed=seed))
    Z2 = tf.nn.conv2d(P1, W2, strides=[1,1,1,1], padding='VALID', name='conv2')
    A2  = tf.nn.relu(Z2, name='relu2')
    print("A2 shape:", A2.shape)
    P2 = tf.nn.max_pool(A2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name='max_pool2')
    print("P2 shape:", P2.shape)
    # 14x14

    # Layer 03 Conv: output feature maps: 120 filters X 5x5
    W3 = tf.get_variable("W3", [kernel_size, kernel_size, n_channels_02, n_channels_03], 
                         initializer=tf.contrib.layers.xavier_initializer(seed=seed))
    Z3 = tf.nn.conv2d(P2, W3, strides=[1,1,1,1], padding='SAME', name='conv3')
    A3  = tf.nn.relu(Z3, name='relu3')
    print("A3 shape:", A3.shape)

    F = tf.contrib.layers.flatten(A3)
    print('A3 flatten shape', F.shape)
    
    Z4 = tf.contrib.layers.fully_connected(F, n_channels_04, activation_fn=None)
    A4 = tf.nn.relu(Z4)
    print("A4 shape:", A4.shape)

    Z5 = tf.contrib.layers.fully_connected(A4, n_channels_05, activation_fn=None)
    print("A5 shape:", Z5.shape)

    prediction = Z5
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=prediction))
    
    # compute trainings error
    cost = tf.losses.sparse_softmax_cross_entropy(labels=y_input, logits=prediction)
    
    # use the Adam optimizer to derive the cost function and update the weights
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # accuracy for multiple batches
    acc = tf.contrib.metrics.accuracy(labels=y_input, predictions=tf.argmax(prediction, axis=-1))

Input shape (?, 32, 32, 3)
A1 shape: (?, 28, 28, 6)
P1 shape: (?, 14, 14, 6)
A2 shape: (?, 10, 10, 16)
P2 shape: (?, 5, 5, 16)
A3 shape: (?, 5, 5, 120)
A3 flatten shape (?, 3000)
A4 shape: (?, 84)
A5 shape: (?, 10)


In [ ]:
# session configuration
config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True

train_costs = []
test_costs = []

# work on GPU if available
with tf.device("/gpu:0"):

    # start a new session
    with tf.Session(graph=graphCNN, config=config) as session:  
    
        # initialize weights and bias variables
        session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     
        
        # which nodes to fetch from the computation graph
        fetch_train_nodes = {
            'cost' : cost,
            'optimizer' : optimizer 
        }
        
        for epoch in range(1):
            
            # shuffle data
            permutation = np.random.permutation(X_train.shape[0])
            X_train = X_train[permutation]
            y_train = y_train[permutation]
            
            
            for i in range(num_iters):
                X_batch = X_train[i * batch_size:(i + 1) * batch_size]
                y_batch = y_train[i * batch_size:(i + 1) * batch_size]
                
                output_batch = session.run(fetch_train_nodes, feed_dict={x_input: X_batch, y_input: y_batch})
                if(i % 100) == 0:
                    train_costs.append(output_batch["cost"])
                    i_test = np.random.randint(X_test.shape[0], size=batch_size)
                    X_batch_test = X_test[i_test]
                    y_batch_test = y_test[i_test]
                    output_batch_test = session.run(fetch_train_nodes, feed_dict={x_input: X_batch_test, y_input: y_batch_test})
                    test_costs.append(output_batch_test["cost"])
                    
                    print("Error [{}/{}] train: {:.4f} test: {:.4f}".format(epoch, i, output_batch["cost"], output_batch_test["cost"]))
                if(i % 500) == 0:
                    print("Train accuracy {:.4f}%".format(session.run(acc, feed_dict={x_input: X_batch, y_input: y_batch})))
                    print("Test accuracy {:.4f}%".format(session.run(acc, feed_dict={x_input: X_batch_test, y_input: y_batch_test})))
                    

        # check against test set
        print("Test accuracy ", session.run(acc, feed_dict={x_input: X_train, y_input: y_train}))
        print("Test accuracy ", session.run(acc, feed_dict={x_input: X_test, y_input: y_test}))

Error [0/0] train: 2.3041 test: 3.1120
Train accuracy 0.1426%
Test accuracy 0.1074%
Error [0/100] train: 2.2947 test: 2.2533
Error [0/200] train: 2.1761 test: 2.1079
Error [0/300] train: 2.3913 test: 1.9258
Error [0/400] train: 1.6969 test: 1.9282
Error [0/500] train: 1.4547 test: 1.8728
Train accuracy 0.1494%
Test accuracy 0.1152%
Error [0/600] train: 1.7435 test: 1.8411
Error [0/700] train: 1.5471 test: 1.7321
Error [0/800] train: 1.5829 test: 2.0068
Error [0/900] train: 1.7403 test: 1.7047
Error [0/1000] train: 1.7484 test: 1.6600
Train accuracy 0.1094%
Test accuracy 0.1133%
Error [0/1100] train: 1.2879 test: 1.5842
Error [0/1200] train: 1.6271 test: 1.9299
Error [0/1300] train: 1.3786 test: 1.3602
Error [0/1400] train: 1.6370 test: 1.8290
Error [0/1500] train: 1.6509 test: 1.7395
Train accuracy 0.0859%
Test accuracy 0.0918%


<hr />
<h2>Abgabe</h2>

<p>Das von Ihnen erstellte Notebook und die PDF Datei m&uuml;ssen sp&auml;testens bis zum 21. Januar 2018 um 23:59 UTC+1 ;) per E-Mail an&nbsp;<a href="mailto:hezel@htw-berlin.de" target="_blank">hezel@htw-berlin.de</a>&nbsp;eingesendet werden. Verwenden Sie als Betreff bitte &quot;CV1718 &Uuml;bung4 &lt;NAME&gt;&quot; und als Notebook Name &quot;CV1718_Ue4_Tensorflow_ConvNet_CIFAR_NAME.ipynb&quot; sowie &quot;CV1718_Ue4_Tensorflow_ConvNet_CIFAR_NAME.pdf&quot; f&uuml;r die PDF. Bevor Sie mir eine Mail schicken, entfernen Sie bitte &uuml;ber &quot;Kernel&quot; -&gt; &quot;Restart and Clear Output&quot; s&auml;mtlichen von Python erstellten Inhalt und speichern anschlie&szlig;end das Notebook &quot;File&quot; -&gt; &quot;Save and Checkpoint&quot;.</p>
